#### Energy Data Machine Modelling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor


### Support Vector Machine (SVM)

In [ ]:
# Re-load the data set
df = pd.read_csv('energydata.csv')

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M', errors='coerce')

# Extract numerical features from the 'date' column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

# Create a copy of the dataframe
df_new = df.copy()

# Reset index of the new dataframe
df_new_reset = df_new.reset_index(drop=True)


In [ ]:
# Prepare the feature matrix and target vector
X = df_new.drop(columns=['Appliances', 'year', 'date'])  # Exclude the target and year column
y = df_new['Appliances']  # Target variable


In [ ]:
# Select top features for training (from MI and RFE)
top_features = ['hour', 'T9', 'T7', 'RH_6', 'T5', 'T4', 'T8', 'T1', 'T3', 'RH_1', 'Press_mm_hg', 'T_out', 'T2', 'RH_8', 'RH_5']

# Get X_train with only the selected features
X_train_df = X[top_features]

# Ensure y matches the shape of X_train_df
y = df_new['Appliances'][:len(X_train_df)]  # Adjust y to have the same number of rows as X_train_df


In [ ]:
# Now split the data, ensuring both X_train_df and y are consistently split

X_train, X_temp, y_train, y_temp = train_test_split(X_train_df, y, test_size=0.3, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Check shapes of the resulting splits
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_eval shape:", X_eval.shape)
print("y_eval shape:", y_eval.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


#### SVM with the radial basis function (RBF) kernel without hyperparameter tuning

In [ ]:
# Scale the features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Scaling for training set
X_eval_scaled = scaler.transform(X_eval)  # Scaling for evaluation set

In [ ]:
# Define the SVM model with RBF kernel (default parameters)
svm_model = SVR(kernel='rbf')

# Train the model
svm_model.fit(X_train_scaled, y_train)

In [ ]:
#Make predictions on the evaluation set
y_eval_pred = svm_model.predict(X_eval_scaled)

#Evaluate the model on the evaluation set
mse = mean_squared_error(y_eval, y_eval_pred)
r2 = r2_score(y_eval, y_eval_pred)

print(f"Mean Squared Error on Evaluation Set: {mse}")
print(f"R-squared on Evaluation Set: {r2}")


- Base SVM got poor result, will now do SVM with tuning

#### SVM Tuning with cv=5 and 18 fits

In [ ]:
#Tuning the SVM with cross-validation and multiple fits

# Set up parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.2],
    'gamma': ['scale', 'auto']
}

# Grid search with cross-validation
grid_search = GridSearchCV(SVR(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and model
print("Best parameters:", grid_search.best_params_)
svr_model = grid_search.best_estimator_

In [ ]:
# Make predictions on the evaluation set
y_pred = svr_model.predict(X_eval_scaled)

# Evaluate the model's performance
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_eval, y_pred)
r2 = r2_score(y_eval, y_pred)

# Output the evaluation metrics
print(f"Mean Squared Error on Evaluation Set: {mse}")
print(f"R-squared on Evaluation Set: {r2}")

In [ ]:
print(grid_search.n_splits_)  # Number of cross-validation splits
print(len(grid_search.cv_results_['mean_test_score']))  # Total number of fits

##### Summary of SVM on the Training Set:
- SVM with kernel='rbf', C=1.0, epsilon=0.1, got poor result with MSE = 8213.612075235185 and r^2 = 0.03: meaning the model is not fitting well.
- SVM is tuned with 5-cross validation and total 18 fits, with  best parameters C'= 10, 'epsilon'= 0.2, 'gamma'= 'scale, slightly improved the result but still not acceptable with MSE = 7659 and 2^ at 0.10.
- Possible reasons could be that SVM struggle with high dimension data (training set at 13,814 rows and 15 columns) and the complex relationship of the features.
- Next step is to explore tree based models that are more capable in handling high dimension and complex data set.

### Tree-Based Models (Random Forest, XGBoost)

#### Random Forest

In [ ]:
# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model with the training data
rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Predict on the evaluation set
y_eval_pred = rf_model.predict(X_eval_scaled)

# Calculate Mean Squared Error (MSE) and R-squared (R²)
mse = mean_squared_error(y_eval, y_eval_pred)
r2 = r2_score(y_eval, y_eval_pred)

print(f"Mean Squared Error on Evaluation Set: {mse}")
print(f"R-squared on Evaluation Set: {r2}")

- Random Forest has improved both MSE and r^2 with RMSE=3,880 and r^=0.55
- Will try to get better resukt with hyperparameter tuning of Random Forest

#### Random Forest with Hyperparameter Tuning: 

In [ ]:
#RF tuning with parammeter distrubution listed below at cv=5, iter=50

# Set up the parameter distribution for Random Forest
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Set up the Randomized Search with 5 cross-validation folds
random_search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42),
                                   param_distributions=param_dist, 
                                   n_iter=50, cv=5, n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(X_train_scaled, y_train)


In [ ]:
# Best parameters and best model
print("Best parameters:", random_search.best_params_)
rf_best_model_random = random_search.best_estimator_

# Evaluate the tuned model
y_eval_pred_random = rf_best_model_random.predict(X_eval_scaled)
mse_random = mean_squared_error(y_eval, y_eval_pred_random)
r2_random = r2_score(y_eval, y_eval_pred_random)

print(f"Mean Squared Error on Evaluation Set (Randomized Tuned): {mse_random}")
print(f"R-squared on Evaluation Set (Randomized Tuned): {r2_random}")

- With best parameters from RandmozedSearchCV of Random Forest, MSE = 3,823, while r^2=0.553. This is a slight improvemen with the base RF model.
- Next will explore further with tree-based model now using XG boost.

#### XG Boost

In [ ]:
# Scale the features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Initialize the model with default parameters
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Train the model on the training set
xgb_model.fit(X_train, y_train)

# Predict on the evaluation set
y_eval_pred = xgb_model.predict(X_eval)

In [ ]:
# Calculate metrics
mse_eval = mean_squared_error(y_eval, y_eval_pred)
r2_eval = r2_score(y_eval, y_eval_pred)

print(f"Mean Squared Error on Evaluation Set: {mse_eval}")
print(f"R-squared on Evaluation Set: {r2_eval}")


#### XG Boost with Reduced Features

In [ ]:
#Further reducing features using Recursive Feauture Elimination (currently selection is 15) to increase MSE and R^2

#Train a Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': top_features, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select top N important features (e.g., top 10)
top_selected_features = feature_importance_df['Feature'].head(10).tolist()

print("Top selected features:", top_selected_features)

In [ ]:
# Reloading the data set to use the 10 features
# Re-load the data set
df = pd.read_csv('energydata.csv')

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M', errors='coerce')

# Extract numerical features from the 'date' column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

# Create a copy of the dataframe
df_new = df.copy()

# Reset index of the new dataframe
df_new_reset = df_new.reset_index(drop=True)
# Prepare the feature matrix and target vector
X = df_new.drop(columns=['Appliances', 'year', 'date'])  # Exclude the target and year column
y = df_new['Appliances']  # Target variable
# Select top features for training (from MI and RFE)
top_features = ['hour', 'Press_mm_hg', 'T_out', 'T3', 'RH_1', 'RH_5', 'T8', 'RH_8', 'RH_6', 'T4']
# Get X_train with only the selected features
X_train_df = X[top_features]

# Ensure y matches the shape of X_train_df
y = df_new['Appliances'][:len(X_train_df)]  # Adjust y to have the same number of rows as X_train_df

# Now split the data, ensuring both X_train_df and y are consistently split
X_train, X_temp, y_train, y_temp = train_test_split(X_train_df, y, test_size=0.3, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Check shapes of the resulting splits
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_eval shape:", X_eval.shape)
print("y_eval shape:", y_eval.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Initialize the model with default parameters
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Train the model on the training set
xgb_model.fit(X_train, y_train)

# Predict on the evaluation set
y_eval_pred = xgb_model.predict(X_eval)

# Calculate metrics
mse_eval = mean_squared_error(y_eval, y_eval_pred)
r2_eval = r2_score(y_eval, y_eval_pred)

print(f"Mean Squared Error on Evaluation Set: {mse_eval}")
print(f"R-squared on Evaluation Set: {r2_eval}")


- Implementing reduced features (using 10 most impt from the re-run Recursive Featue elimination) did not greatly improved the MSE and r^2.
- Next approcah is tune the XG boost with GridsearchCV.

In [ ]:

# Define the parameter grid
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [1, 1.5, 2, 5]
}

# Initialize the XGBoost regressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter combinations to try
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit on training data
random_search.fit(X_train, y_train)

In [ ]:
# Get the best model and parameters
best_xgb = random_search.best_estimator_
print("Best parameters:", random_search.best_params_)

# Evaluate on the evaluation set
y_pred_eval = best_xgb.predict(X_eval)
mse_eval = mean_squared_error(y_eval, y_pred_eval)
r2_eval = r2_score(y_eval, y_pred_eval)

print("Mean Squared Error on Evaluation Set (Tuned XGBoost):", mse_eval)
print("R-squared on Evaluation Set (Tuned XGBoost):", r2_eval)

In [ ]:
# Get the best model and parameters
best_xgb = random_search.best_estimator_
print("Best parameters:", random_search.best_params_)

# Evaluate on the evaluation set
y_pred_eval = best_xgb.predict(X_eval)
mse_eval = mean_squared_error(y_eval, y_pred_eval)
r2_eval = r2_score(y_eval, y_pred_eval)

print("Mean Squared Error on Evaluation Set (Tuned XGBoost):", mse_eval)
print("R-squared on Evaluation Set (Tuned XGBoost):", r2_eval)


In [ ]:
#further tuning XG-boost

param_grid = {
    'n_estimators': [400, 500, 600],
    'learning_rate': [0.05, 0.08, 0.1, 0.12],
    'max_depth': [8, 9, 10, 11, 12],
    'subsample': [0.5, 0.6, 0.7],
    'colsample_bytree': [0.5, 0.6, 0.7],
    'gamma': [0.1, 0.2, 0.3],
    'reg_alpha': [0.4, 0.5, 0.6],
    'reg_lambda': [4, 5, 6]
}

random_search = RandomizedSearchCV(
    estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Fewer iterations but in a smaller range
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

In [ ]:
# Get the best model
best_xgb = random_search.best_estimator_
print("New Best parameters:", random_search.best_params_)

# Evaluate again
y_pred_eval = best_xgb.predict(X_eval)
mse_eval = mean_squared_error(y_eval, y_pred_eval)
r2_eval = r2_score(y_eval, y_pred_eval)

print("Fine-Tuned Mean Squared Error:", mse_eval)
print("Fine-Tuned R-squared:", r2_eval)


In [ ]:
#Before further continuing with XG-boost tuning, taking a look at the residual p`attern of y_train predict.

# Predict on the training data
y_train_pred = best_xgb.predict(X_train)

# Calculate residuals for the training data
residuals_train = y_train - y_train_pred

# Plot residuals vs predicted values (training data)
plt.figure(figsize=(10, 6))
plt.scatter(y_train_pred, residuals_train, color='blue', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Line at y=0
plt.title('Residuals vs. Predicted Values (Training Data)')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# the residuals seems to forma pattern with heavy dense dots from 0-400 at X-axis while showing slight 
#randomness at 600 predicted values. From here, resuals are scattered at 0-50 range.
#This could mean that the model may be underfitting or failing to capture certain relationships.

In [ ]:
# Histogram of the residuals (training data)
plt.figure(figsize=(10, 6))
sns.histplot(residuals_train, kde=True, bins=30, color='blue')
plt.title('Histogram of Residuals (Training Data)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

#The histogram of the residuals has almost normal distrubution centered at 0 , but slightly skewed to the left.

In [ ]:
import scipy.stats as stats
#Further looking in to the residuals
# Q-Q plot (training data residuals)
plt.figure(figsize=(10, 6))
stats.probplot(residuals_train, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (Training Data)')
plt.show()

#the heavy tails (downward and upward direction) of the Q-Q plot may indicate problematic outliers that could distort model performance).

- The residual plot summary indicates that:
  1. residual scatter plot, not random = adjust the model or do further feature engineering.
  2. histogram = centered at 0 but bins are not uniform on each side, showing slight skewed to the left, thus suggeting that the model is systematically underestimating the lower values of the target variable.
  3. One tail upward, suggesting outleiers and downward on the other suggesting  skewness.

- From these results, we will apply log transformation to address skweness. If this does not work, we will re-visit the features. Residuals forming pattern and not randomly scettrred could mean underfitting or missing features.

In [ ]:
# Apply log transformation to the target variable
y_train_log = np.log(y_train + 1)  # Adding 1 to avoid log(0) if there are any zeros in y_train

# Define the XGBoost model with your best hyperparameters
model = XGBRegressor(
    subsample=0.6,
    reg_lambda=6,
    reg_alpha=0.4,
    n_estimators=600,
    max_depth=12,
    learning_rate=0.08,
    gamma=0.2,
    colsample_bytree=0.6
)

# Train the model on the transformed target variable
model.fit(X_train, y_train_log)

# Predict on the training data using the log-transformed model
y_train_pred_log = model.predict(X_train)

# Calculate residuals for the log-transformed data
residuals_log = y_train_log - y_train_pred_log

In [ ]:
# Plot residuals vs predicted values (log-transformed target)
plt.figure(figsize=(10, 6))
plt.scatter(y_train_pred_log, residuals_log, color='blue', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Line at y=0
plt.title('Residuals vs. Predicted Values (Log-transformed Target)')
plt.xlabel('Predicted Values (Log)')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Histogram of the residuals (log-transformed target)
plt.figure(figsize=(10, 6))
sns.histplot(residuals_log, kde=True, bins=30, color='blue')
plt.title('Histogram of Residuals (Log-transformed Target)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Q-Q plot (log-transformed target residuals)
plt.figure(figsize=(10, 6))
stats.probplot(residuals_log, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (Log-transformed Target)')
plt.show()

In [ ]:
# Handle the cyclic hour feature by creating hour_sin and hour_cos
X_train['hour_sin'] = np.sin(2 * np.pi * X_train['hour'] / 24)  # Create cyclic feature for hour
X_train['hour_cos'] = np.cos(2 * np.pi * X_train['hour'] / 24)  # Create cyclic feature for hour

# Same for evaluation set
X_eval['hour_sin'] = np.sin(2 * np.pi * X_eval['hour'] / 24)
X_eval['hour_cos'] = np.cos(2 * np.pi * X_eval['hour'] / 24)

# Replace zeros with a small constant (1e-5) in the log-transformed columns before applying log1p
log_columns = X_train.columns.difference(['hour_sin', 'hour_cos'])  # Exclude hour_sin and hour_cos
X_train[log_columns] = X_train[log_columns].replace(0, 1e-5)  # Replace zeros in training set
X_eval[log_columns] = X_eval[log_columns].replace(0, 1e-5)    # Replace zeros in evaluation set

# Handle negative values by replacing them with a small constant (1e-5)
X_train[log_columns] = X_train[log_columns].where(X_train[log_columns] >= 0, 1e-5)  # Handle negative values
X_eval[log_columns] = X_eval[log_columns].where(X_eval[log_columns] >= 0, 1e-5)    # Handle negative values

# Apply log transformation to non-cyclic features
log_columns = X_train.columns.difference(['hour_sin', 'hour_cos'])  # Exclude hour_sin and hour_cos
X_train_log = X_train.copy()
X_train_log[log_columns] = np.log1p(X_train[log_columns])  # Log transformation for non-cyclic columns

# Apply the same log transformation to the evaluation set
X_eval_log = X_eval.copy()
X_eval_log[log_columns] = np.log1p(X_eval[log_columns])  # Log transformation for non-cyclic columns

# Shift the target variable to avoid log(0)
y_train_log = np.log1p(y_train)  # Apply log1p to target variable y_train
y_eval_log = np.log1p(y_eval)    # Apply log1p to target variable y_eval


In [ ]:
# Initialize the XGBoost model

model = xgb.XGBRegressor(
    subsample=0.6,
    reg_lambda=6,
    reg_alpha=0.4,
    n_estimators=600,
    max_depth=12,
    learning_rate=0.08,
    gamma=0.2,
    colsample_bytree=0.6
)
# Train the model on log-transformed training data
model.fit(X_train_log, y_train_log)

In [ ]:
# Make predictions on the evaluation data
y_eval_pred = model.predict(X_eval_log)

# Evaluate the model performance on the evaluation set
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

# Print the evaluation results
print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

#The result has better MSE and r^2, from this we can tune the model further

In [ ]:
#further tuning of XG Boost

#Define the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameters grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),           # Number of trees in boosting rounds
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Step size shrinking
    'max_depth': [3, 5, 7, 9, 10],                        # Maximum depth of the tree
    'min_child_weight': [1, 2, 3, 4, 5],                  # Minimum sum of instance weight
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],              # Fraction of training samples for each tree
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],       # Fraction of features for each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],                    # Minimum loss reduction
    'scale_pos_weight': [1, 2, 3],                        # Scale the positive weight (useful for imbalanced data)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

In [ ]:
# Evaluate on training set
y_train_pred = best_xgb_model.predict(X_train_log)
y_eval_pred = best_xgb_model.predict(X_eval_log)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_train = r2_score(y_train_log, y_train_pred)

# Print results
print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")

In [ ]:
# Predict on training and evaluation sets
y_train_pred = best_xgb_model.predict(X_train_log)
y_eval_pred = best_xgb_model.predict(X_eval_log)

# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Evaluation set plot
axs[1].scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6)
axs[1].plot([y_eval_log.min(), y_eval_log.max()], [y_eval_log.min(), y_eval_log.max()], color='red', linestyle='--')
axs[1].set_title("Evaluation Set: True vs Predicted")
axs[1].set_xlabel("True values (y_eval_log)")
axs[1].set_ylabel("Predicted values (y_eval_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training and Evaluation Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

#### The tuned XGBoost showed overfitting as we can see from this  result:
- Evaluation Set MSE: 0.19057607821743702
- Evaluation Set R-squared: 0.5215629315035697
- Training Set MSE: 0.06139894743900729
- Training Set R-squared: 0.8532941635612186

- To address overfitting, lets try to
  1. reduce depth
  2. lower n_setimators
  3. experiment on cross-validation

- XGBoost Hyper Parameter Tuning, reducing n_estimators, max_dept, and increase cv =10

In [ ]:
#Define the model

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameters grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(50, 75 , 100),           # Number of trees in boosting rounds
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Step size shrinking
    'max_depth': [2, 4, 5, 6, 7],                        # Maximum depth of the tree
    'min_child_weight': [1, 2, 3, 4, 5],                  # Minimum sum of instance weight
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],              # Fraction of training samples for each tree
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],       # Fraction of features for each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],                    # Minimum loss reduction
}    

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=10, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

In [ ]:
# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")

In [ ]:
# Predict on training and evaluation sets
y_train_pred = best_xgb_model.predict(X_train_log)
y_eval_pred = best_xgb_model.predict(X_eval_log)

# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Evaluation set plot
axs[1].scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6)
axs[1].plot([y_eval_log.min(), y_eval_log.max()], [y_eval_log.min(), y_eval_log.max()], color='red', linestyle='--')
axs[1].set_title("Evaluation Set: True vs Predicted")
axs[1].set_xlabel("True values (y_eval_log)")
axs[1].set_ylabel("Predicted values (y_eval_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training and Evaluation Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Further tuning of hyperparameters, cv=10, n=estimators reduced, 

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameters grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(300, 600 , 900),           # Number of trees in boosting rounds
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Step size shrinking
    'max_depth': [2, 4, 5, 6, 7],                        # Maximum depth of the tree
    'min_child_weight': [1, 2, 3, 4, 5],                  # Minimum sum of instance weight
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],              # Fraction of training samples for each tree
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],       # Fraction of features for each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],                    # Minimum loss reduction
}    

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=10, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

In [ ]:
# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")

# Still slightly overfitting, tuning further

In [ ]:
# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Evaluation set plot
axs[1].scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6)
axs[1].plot([y_eval_log.min(), y_eval_log.max()], [y_eval_log.min(), y_eval_log.max()], color='red', linestyle='--')
axs[1].set_title("Evaluation Set: True vs Predicted")
axs[1].set_xlabel("True values (y_eval_log)")
axs[1].set_ylabel("Predicted values (y_eval_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training and Evaluation Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#still getting an overfitting issue, and to address it. we will try to removed redundant features, reduce max_dept/estimators, and incrase subsample

# Get feature importance scores from the trained model
feature_importance = random_search.best_estimator_.feature_importances_

# Convert to DataFrame for better visualization
importances_df = pd.DataFrame({'Feature': X_train_log.columns, 'Importance': feature_importance})
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'], importances_df['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('XGBoost Feature Importance')
plt.gca().invert_yaxis()  # Highest importance at the top
plt.show()

In [ ]:
#Running the whole code again of log transformmation then remove the hour as redundant feature

# Handle the cyclic hour feature by creating hour_sin and hour_cos
X_train['hour_sin'] = np.sin(2 * np.pi * X_train['hour'] / 24)  # Create cyclic feature for hour
X_train['hour_cos'] = np.cos(2 * np.pi * X_train['hour'] / 24)  # Create cyclic feature for hour

# Same for evaluation set
X_eval['hour_sin'] = np.sin(2 * np.pi * X_eval['hour'] / 24)
X_eval['hour_cos'] = np.cos(2 * np.pi * X_eval['hour'] / 24)

# Drop the 'hour' column after creating the cyclic features
X_train = X_train.drop(columns=['hour'])
X_eval = X_eval.drop(columns=['hour'])

# Replace zeros with a small constant (1e-5) in the log-transformed columns before applying log1p
log_columns = X_train.columns.difference(['hour_sin', 'hour_cos'])  # Exclude hour_sin and hour_cos
X_train[log_columns] = X_train[log_columns].replace(0, 1e-5)  # Replace zeros in training set
X_eval[log_columns] = X_eval[log_columns].replace(0, 1e-5)    # Replace zeros in evaluation set

# Handle negative values by replacing them with a small constant (1e-5)
X_train[log_columns] = X_train[log_columns].where(X_train[log_columns] >= 0, 1e-5)  # Handle negative values
X_eval[log_columns] = X_eval[log_columns].where(X_eval[log_columns] >= 0, 1e-5)    # Handle negative values

# Apply log transformation to non-cyclic features
log_columns = X_train.columns.difference(['hour_sin', 'hour_cos'])  # Exclude hour_sin and hour_cos
X_train_log = X_train.copy()
X_train_log[log_columns] = np.log1p(X_train[log_columns])  # Log transformation for non-cyclic columns

# Apply the same log transformation to the evaluation set
X_eval_log = X_eval.copy()
X_eval_log[log_columns] = np.log1p(X_eval[log_columns])  # Log transformation for non-cyclic columns

# Shift the target variable to avoid log(0)
y_train_log = np.log1p(y_train)  # Apply log1p to target variable y_train
y_eval_log = np.log1p(y_eval)    # Apply log1p to target variable y_eval

In [ ]:
# Check if 'hour' has been dropped. 
print("X_train_log columns:", X_train_log.columns)
print("X_eval_log columns:", X_eval_log.columns)

In [ ]:
#Initialize the XGboost with tuned hyperparameter

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameters grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(200, 400, 600),           # Number of trees in boosting rounds
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Step size shrinking
    'max_depth': [2, 3, 4, 5, 6],                        # Maximum depth of the tree
    'min_child_weight': [1, 2, 3, 4, 5],                  # Minimum sum of instance weight
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],              # Fraction of training samples for each tree
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],       # Fraction of features for each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],                    # Minimum loss reduction
                            
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=15, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")


In [ ]:
#Further tuning of hyper parameters

# Updated parameter grid with more refined values
param_dist = {
    'n_estimators': np.arange(300, 501, 100),  # Increase n_estimators range
    'learning_rate': [0.05, 0.1],               # Lower learning rate
    'max_depth': [4, 5],                        # Try smaller max_depth
    'min_child_weight': [5, 6],                  # Increase min_child_weight slightly
    'subsample': [0.7, 0.8],                    # Try slightly higher subsample
    'colsample_bytree': [0.7, 0.8],             # Try slightly higher colsample
    'gamma': [0, 0.1, 0.2],                     # Add some regularization with gamma
}

# RandomizedSearchCV with refined parameters
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=10, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")


In [ ]:
# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Evaluation set plot
axs[1].scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6)
axs[1].plot([y_eval_log.min(), y_eval_log.max()], [y_eval_log.min(), y_eval_log.max()], color='red', linestyle='--')
axs[1].set_title("Evaluation Set: True vs Predicted")
axs[1].set_xlabel("True values (y_eval_log)")
axs[1].set_ylabel("Predicted values (y_eval_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training and Evaluation Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#More tuning as the result is still overfitting

param_dist = {
    'n_estimators': np.arange(200, 401, 100),  # Decrease n_estimators range
    'learning_rate': [0.05, 0.1],               # Continue with small learning rates
    'max_depth': [3, 4],                        # Reduce max_depth further
    'min_child_weight': [6, 7],                  # Increase min_child_weight slightly
    'subsample': [0.7, 0.8],                    # Keep subsample around 0.7 or 0.8
    'colsample_bytree': [0.7, 0.8],             # Keep colsample_bytree moderate
    'gamma': [0.1, 0.2],                        # Increase gamma for more regularization
}

# RandomizedSearchCV with refined parameters
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=15, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model to the training data
random_search.fit(X_train_log, y_train_log)

# Print the best parameters and the corresponding score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")


In [ ]:
# Define the hyperparameters grid for RandomizedSearchCV with fewer parameters
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Restrict to lower values
    'max_depth': [3, 5, 7],                    # Try fewer depth levels
    'subsample': [0.6, 0.7, 0.8],              # Range for subsample
    'colsample_bytree': [0.6, 0.7],            # Fraction of features to use
}

# Initialize RandomizedSearchCV with fewer iterations and folds
random_search = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=param_dist, 
    n_iter=15,  # Reduced n_iter
    cv=5,       # Lower cv back to 5
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

# Fit the randomized search to the training data
random_search.fit(X_train_log, y_train_log)

# Print the best hyperparameters and the best cross-validation score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the evaluation set
y_eval_pred = best_xgb_model.predict(X_eval_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Calculate MSE and R-squared for the evaluation set
from sklearn.metrics import mean_squared_error, r2_score
mse_eval = mean_squared_error(y_eval_log, y_eval_pred)
r2_eval = r2_score(y_eval_log, y_eval_pred)

mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_eval = r2_score(y_train_log, y_train_pred)

print(f"Evaluation Set MSE: {mse_eval}")
print(f"Evaluation Set R-squared: {r2_eval}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")


In [ ]:
# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Evaluation set plot
axs[1].scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6)
axs[1].plot([y_eval_log.min(), y_eval_log.max()], [y_eval_log.min(), y_eval_log.max()], color='red', linestyle='--')
axs[1].set_title("Evaluation Set: True vs Predicted")
axs[1].set_xlabel("True values (y_eval_log)")
axs[1].set_ylabel("Predicted values (y_eval_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_eval_log.min()), max(y_train_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training and Evaluation Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

- At last! This will be used to evaluate my test set.

In [ ]:
#Running the whole code again of log transformmation for the X_test

# Handle the cyclic hour feature by creating hour_sin and hour_cos (before dropping the 'hour' column)
X_test['hour_sin'] = np.sin(2 * np.pi * X_test['hour'] / 24)  # Create cyclic feature for hour
X_test['hour_cos'] = np.cos(2 * np.pi * X_test['hour'] / 24)  # Create cyclic feature for hour

# Drop the 'hour' column after creating the cyclic features (both in train and test sets)
X_test = X_test.drop(columns=['hour'])

# Replace zeros with a small constant (1e-5) in the log-transformed columns before applying log1p
X_test[log_columns] = X_test[log_columns].replace(0, 1e-5)    # Replace zeros in test set

# Handle negative values by replacing them with a small constant (1e-5)
X_test[log_columns] = X_test[log_columns].where(X_test[log_columns] >= 0, 1e-5)    # Handle negative values

# Apply the same log transformation to the test set
X_test_log = X_test.copy()
X_test_log[log_columns] = np.log1p(X_test_log[log_columns])  # Log transformation for non-cyclic columns

# Shift the target variable to avoid log(0)
y_train_log = np.log1p(y_train)  # Apply log1p to target variable y_train
y_test_log = np.log1p(y_test)    # Apply log1p to target variable y_test


In [ ]:
# Define the hyperparameters grid for RandomizedSearchCV with fewer parameters
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Restrict to lower values
    'max_depth': [3, 5, 7],                    # Try fewer depth levels
    'subsample': [0.6, 0.7, 0.8],              # Range for subsample
    'colsample_bytree': [0.6, 0.7],            # Fraction of features to use
}

# Initialize RandomizedSearchCV with fewer iterations and folds
random_search = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=param_dist, 
    n_iter=15,  # Reduced n_iter
    cv=5,       # Lower cv back to 5
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

In [ ]:
# Fit the randomized search to the training data
random_search.fit(X_train_log, y_train_log)

In [ ]:
# Print the best hyperparameters and the best cross-validation score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-Validation Score: {random_search.best_score_}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_xgb_model = random_search.best_estimator_

# Predict on the test set and training set
y_test_pred = best_xgb_model.predict(X_test_log)
y_train_pred = best_xgb_model.predict(X_train_log)

# Test set MSE and R-squared
mse_test = mean_squared_error(y_test_log, y_test_pred)
r2_test = r2_score(y_test_log, y_test_pred)

# Training set MSE and R-squared
mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_train = r2_score(y_train_log, y_train_pred)

# Print results
print(f"Test Set MSE: {mse_test}")
print(f"Test Set R-squared: {r2_test}")

print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")

In [ ]:
# Plotting the predictions against the true values for training and evaluation sets
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Training set plot
axs[0].scatter(y_train_log, y_train_pred, color='blue', alpha=0.6)
axs[0].plot([y_train_log.min(), y_train_log.max()], [y_train_log.min(), y_train_log.max()], color='red', linestyle='--')
axs[0].set_title("Training Set: True vs Predicted")
axs[0].set_xlabel("True values (y_train_log)")
axs[0].set_ylabel("Predicted values (y_train_pred)")
axs[0].grid(True)

# Test set plot
axs[1].scatter(y_test_log, y_test_pred, color='green', alpha=0.6)
axs[1].plot([y_test_log.min(), y_test_log.max()], [y_test_log.min(), y_test_log.max()], color='red', linestyle='--')
axs[1].set_title("Test Set: True vs Predicted")
axs[1].set_xlabel("True values (y_test_log)")
axs[1].set_ylabel("Predicted values (y_test_pred)")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting both training and evaluation set predictions together
plt.figure(figsize=(10, 6))

# Plot training set
plt.scatter(y_train_log, y_train_pred, color='blue', alpha=0.6, label='Training Set', s=20)

# Plot evaluation set
plt.scatter(y_eval_log, y_eval_pred, color='green', alpha=0.6, label='Evaluation Set', s=20)

# Plot test set
plt.scatter(y_test_log, y_test_pred, color='pink', alpha=0.6, label='Test Set', s=20)

# Plot a line for perfect prediction (y_true = y_pred)
plt.plot([min(y_train_log.min(), y_test_log.min(), y_eval_log.min()), 
          max(y_train_log.max(), y_test_log.max(), y_eval_log.max())],
         [min(y_train_log.min(), y_test_log.min(), y_eval_log.min()), 
          max(y_train_log.max(), y_test_log.max(), y_eval_log.max())],
         color='red', linestyle='--', label='Perfect Prediction')

# Set labels and title
plt.title("True vs Predicted Values for Training, Evaluation, and Test Sets")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Create a dictionary with the values
data = {
    'Dataset': ['Test Set', 'Training Set', 'Evaluation Set'],
    'MSE': [0.2664, 0.0127, 0.2480],
    'R-squared': [0.4125, 0.9697, 0.9697]
}

# Convert the dictionary to a DataFrame
metrics_table = pd.DataFrame(data)
metrics_table

 ### Plotting the Learning Curve

In [ ]:
# Plotting the Learning Curve

import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Define the best hyperparameters based on your search
best_params = {
    'subsample': 0.7,
    'n_estimators': 400,
    'max_depth': 7,
    'learning_rate': 0.2,
    'colsample_bytree': 0.6
}

# Initialize the XGBoost model with the best parameters
best_xgb_model = xgb.XGBRegressor(
    subsample=best_params['subsample'],
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    colsample_bytree=best_params['colsample_bytree'],
    random_state=42
)

# Fit the model
best_xgb_model.fit(X_train_log, y_train_log)

# Predict on the training and test set
y_train_pred = best_xgb_model.predict(X_train_log)
y_test_pred = best_xgb_model.predict(X_test_log)

# Calculate MSE and R-squared for the test and training set
mse_train = mean_squared_error(y_train_log, y_train_pred)
r2_train = r2_score(y_train_log, y_train_pred)

mse_test = mean_squared_error(y_test_log, y_test_pred)
r2_test = r2_score(y_test_log, y_test_pred)

# Print results
print(f"Training Set MSE: {mse_train}")
print(f"Training Set R-squared: {r2_train}")
print(f"Test Set MSE: {mse_test}")
print(f"Test Set R-squared: {r2_test}")

# Plot the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    best_xgb_model, X_train_log, y_train_log, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Calculate the mean and standard deviation for the train and test scores
train_mean = -train_scores.mean(axis=1)  # Convert negative MSE back to positive
test_mean = -test_scores.mean(axis=1)    # Convert negative MSE back to positive
train_std = train_scores.std(axis=1)
test_std = test_scores.std(axis=1)

# Plot learning curves for MSE
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training error', color='blue')
plt.plot(train_sizes, test_mean, label='Test error', color='green')

# Plot the shaded area for the standard deviation
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)

# Add labels and title
plt.title('Learning Curve with Best Hyperparameters')
plt.xlabel('Number of training samples')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

- Investigating the wide gap b/w test set and train set.

In [ ]:
#Checking for data leakage

#Check if there is any overlap between training and test sets
overlap = X_train.index.intersection(X_test.index)
if len(overlap) > 0:
    print(f"Warning: There is overlap in the training and test sets. Overlapping samples: {len(overlap)}")
else:
    print("No overlap between training and test sets.")

In [ ]:
# Calculate correlation between features in X_train and the target y_train
X_train['Appliances'] = y_train  # Add Appliances to X_train temporarily for correlation
correlation_matrix = X_train.corr()
print(correlation_matrix['Appliances'])

#There is no specific feature which is highly correlated with the target.
# This means no indication of data leakage.

In [ ]:
from sklearn.metrics import mean_squared_error
#Transforming back to original scale and computing for actual energy of the predicted:

# Convert Predictions and Actual Values Back to Original Scale
y_train_pred_orig = np.exp(y_train_pred)  # Convert predictions back
y_test_pred_orig = np.exp(y_test_pred)
y_eval_pred_orig = np.exp(y_eval_pred)  # If you have an evaluation set

y_train_orig = np.exp(y_train_log)  # Convert actual values back
y_test_orig = np.exp(y_test_log)
y_eval_orig = np.exp(y_eval_log)  # If applicable

# Compute MSE in Original Scale
train_mse_orig = mean_squared_error(y_train_orig, y_train_pred_orig)
test_mse_orig = mean_squared_error(y_test_orig, y_test_pred_orig)
eval_mse_orig = mean_squared_error(y_eval_orig, y_eval_pred_orig)

print(f"Training MSE (original scale): {train_mse_orig}")
print(f"Test MSE (original scale): {test_mse_orig}")
print(f"Evaluation MSE (original scale): {eval_mse_orig}")

# Compute Average Difference (to check energy savings)
train_diff = y_train_orig - y_train_pred_orig
test_diff = y_test_orig - y_test_pred_orig
eval_diff = y_eval_orig - y_eval_pred_orig  # If applicable

print(f"Average Training Difference: {train_diff.mean()}")
print(f"Average Test Difference: {test_diff.mean()}")
print(f"Average Evaluation Difference: {eval_diff.mean()}")

# Create a DataFrame for Comparison
comparison_df = pd.DataFrame({
    "Actual Energy (Test)": y_test_orig,
    "Predicted Energy (Test)": y_test_pred_orig,
    "Difference": test_diff
})

print("\nSample of Actual vs Predicted Energy Consumption:")
print(comparison_df.head())  # Show first few rows


In [ ]:
# Total Predicted Energy

total_predicted_energy = df['Predicted Energy'].sum()

#### The MSE result has been transformed back from log to compare with the actual appliance energy consumption.  Hence, below are limitations so it cannot be achieved:

1. The r^2 predicting only 68% of the actual, meaning it has 32% chance of incorrect prediction. This result is not agreeable especially when predicting cost.
2. This will be the same for the total green house gas emission.